In [3]:
from ArticlesRep import MeanSimilarityoneindustry,MeanSimilaritytwoindustry #common function

import pandas as pd

import numpy as np

%matplotlib inline

import pickle

import visdom

from tqdm import tnrange, tqdm_notebook
from tqdm.autonotebook import tqdm
tqdm.pandas()

from time import time,sleep
from datetime import datetime

from sklearn.metrics.pairwise import cosine_similarity
# import visdom
# vis=visdom.Visdom()
# env="TagBased"

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# %matplotlib inline

import re

import sys
from pprint import pprint
#coding:utf-8
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
#有中文出现的情况，需要u'内容'
list_industry=["水泥","食品飲料","石化","紡織","電機機械","電器電纜","化學工業",
               "建材居家用品","造紙","鋼鐵金屬","車輛相關","科技相關","營建地產","運輸","觀光休閒娛樂",
               "金融相關","百貨通路","公用事業","控股","生技醫療保健","農林漁牧","航天軍工","能源","傳播出版","綜合",
               "傳產其他","其他","金屬礦採選",
              ]
small_list_insustry=["車輛相關","生技醫療保健","營建地產","百貨通路","傳播出版"]

C:\Users\garyyu\AppData\Local\Continuum\anaconda3\envs\deeplearningcpu\lib\site-packages\tqdm\autonotebook\__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [180]:
picksample=np.load("D:3.AutoencoderForArticle/picksample.npy")
len(picksample)

12000

In [181]:
bow=np.load("D:3.AutoencoderForArticle/BOW_binary_v01.npy")

In [182]:
with open("D:3.AutoencoderForArticle/dict_collect_industry.p","rb") as f:
    dict_collect_industry=pickle.load(f)

In [193]:
dict_={k:len(dict_collect_industry[k]) for k in dict_collect_industry}
pd.Series(dict_).sort_values(ascending=False)

科技相關      6499
金融相關      1224
鋼鐵金屬      1030
車輛相關       392
生技醫療保健     341
營建地產       322
百貨通路       301
傳播出版       288
電機機械       214
能源         204
傳產其他       172
觀光休閒娛樂     166
紡織         145
運輸         117
化學工業        88
石化          86
公用事業        80
食品飲料        70
農林漁牧        66
造紙          55
水泥          49
建材居家用品      43
電器電纜        24
航天軍工        18
綜合           6
金屬礦採選        0
控股           0
其他           0
dtype: int64

In [223]:
train_test_dict_collect_industry=dict()
for k in dict_collect_industry:
    split=int(len(dict_collect_industry[k])*0.8)
    train_test_dict_collect_industry["train_{}".format(k)]=dict_collect_industry[k][:split]
    train_test_dict_collect_industry["test_{}".format(k)]=dict_collect_industry[k][split:] 

In [224]:
dict_={k:len(train_test_dict_collect_industry[k]) for k in train_test_dict_collect_industry}
pd.Series(dict_).sort_values(ascending=False)

train_科技相關      5199
test_科技相關       1300
train_金融相關       979
train_鋼鐵金屬       824
train_車輛相關       313
train_生技醫療保健     272
train_營建地產       257
test_金融相關        245
train_百貨通路       240
train_傳播出版       230
test_鋼鐵金屬        206
train_電機機械       171
train_能源         163
train_傳產其他       137
train_觀光休閒娛樂     132
train_紡織         116
train_運輸          93
test_車輛相關         79
train_化學工業        70
test_生技醫療保健       69
train_石化          68
test_營建地產         65
train_公用事業        64
test_百貨通路         61
test_傳播出版         58
train_食品飲料        56
train_農林漁牧        52
train_造紙          44
test_電機機械         43
test_能源           41
train_水泥          39
test_傳產其他         35
train_建材居家用品      34
test_觀光休閒娛樂       34
test_紡織           29
test_運輸           24
train_電器電纜        19
test_化學工業         18
test_石化           18
test_公用事業         16
test_食品飲料         14
train_航天軍工        14
test_農林漁牧         14
test_造紙           11
test_水泥           10
test_建材居家用品        9
test_電器電纜          5
test_航天軍工    

In [225]:
# #Save train_test_dict_collect_industry
# with open("D:3.AutoencoderForArticle/train_test_dict_collect_industry.p","wb") as f:
#     pickle.dump(file=f,obj=train_test_dict_collect_industry)

In [244]:
small_list_insustry=["車輛相關","生技醫療保健","營建地產","百貨通路","傳播出版"]

In [236]:
small_train_test_dict_collect_industry=dict()
for i in test_insustry:
    small_train_test_dict_collect_industry["train_{}".format(i)]=train_test_dict_collect_industry["train_{}".format(i)]
    small_train_test_dict_collect_industry["test_{}".format(i)]=train_test_dict_collect_industry["test_{}".format(i)]
    

In [243]:
triplet_train_test_dict_collect_industry=dict()
for i in test_insustry:
    triplet_train_test_dict_collect_industry["train_{}".format(i)]=[]
    triplet_train_test_dict_collect_industry["test_{}".format(i)]=[]

In [240]:
negative=dict()
for k in small_train_test_dict_collect_industry:
    print(k[:5])

{'test_傳播出版': [],
 'test_營建地產': [],
 'test_生技醫療保健': [],
 'test_百貨通路': [],
 'test_車輛相關': [],
 'train_傳播出版': [],
 'train_營建地產': [],
 'train_生技醫療保健': [],
 'train_百貨通路': [],
 'train_車輛相關': []}

In [246]:
small_dict_collect_industry={k:dict_collect_industry[k] for k in small_list_insustry}

In [294]:
negative_pool=dict({k:[] for k in small_dict_collect_industry })
for k in small_dict_collect_industry:
    for j in small_dict_collect_industry:
        if k!=j:negative_pool[k]+=small_dict_collect_industry[j]
    if len(negative_pool[k])!=0:negative_pool[k]=np.random.permutation(negative_pool[k])

## Build dict train/test industry:index in 50,000 single industry VIP news  

In [8]:
with open("D:3.AutoencoderForArticle/DataSet_vip_single_industry","rb") as f:
    Dataset=pickle.load(f)

In [10]:
Dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154884 entries, 0 to 154883
Data columns (total 3 columns):
guid                           154884 non-null object
title_token_plus_body_token    154884 non-null object
indusrty_tags                  154884 non-null object
dtypes: object(3)
memory usage: 3.5+ MB


#### build dict_collect_industry_50000

In [121]:
np.random.seed(0)
suffleindex=np.random.permutation(len(Dataset))
pick=suffleindex[:50000]
pick

array([ 54694, 107229,  12492, ...,  89717,  39250, 126195])

In [122]:
dict_collect_industry_50000={i:[] for i in list_industry }
for i,j in Dataset.loc[pick].iterrows():
    for k in dict_collect_industry_50000:
        if k in j[2]:
            dict_collect_industry_50000[k].append(i)
            break

In [123]:
dict_count={}
for k in dict_collect_industry_50000:
    dict_count[k]=len(dict_collect_industry_50000[k])
df=pd.Series(dict_count)
df.sort_values(ascending=False)

科技相關      27343
金融相關       5037
鋼鐵金屬       4330
車輛相關       1436
營建地產       1337
生技醫療保健     1289
百貨通路       1284
傳播出版       1246
能源          978
電機機械        858
傳產其他        717
觀光休閒娛樂      698
紡織          526
運輸          503
化學工業        387
公用事業        345
食品飲料        309
石化          284
水泥          237
農林漁牧        233
造紙          218
建材居家用品      163
電器電纜        122
航天軍工         72
綜合           44
控股            4
金屬礦採選         0
其他            0
dtype: int64

### build train and test

In [172]:
train_dict_collect_industry_50000=dict()
test_dict_collect_industry_50000=dict()
np.random.seed(10)
rate=3/4
for i in dict_collect_industry_50000:
    induslist=np.random.permutation(dict_collect_industry_50000[i])
    if 0!=len(induslist):
        split=int(np.ceil(len(induslist)*rate))
        train_dict_collect_industry_50000[i]=induslist[:split]
        test_dict_collect_industry_50000[i]=induslist[split:]

#### Save train_test_dict_collect_industry

In [176]:
with open("D:3.AutoencoderForArticle/train_dict_collect_industry_50000.p","wb") as f:
    pickle.dump(file=f,obj=train_dict_collect_industry_50000)
with open("D:3.AutoencoderForArticle/test_dict_collect_industry_50000.p","wb") as f:
    pickle.dump(file=f,obj=test_dict_collect_industry_50000)

In [173]:
dict_count={}
dict_idu_list=train_dict_collect_industry_50000
for k in dict_idu_list:
    dict_count[k]=len(dict_idu_list[k])
df=pd.Series(dict_count)
df.sort_values(ascending=False)

科技相關      20508
金融相關       3778
鋼鐵金屬       3248
車輛相關       1077
營建地產       1003
生技醫療保健      967
百貨通路        963
傳播出版        935
能源          734
電機機械        644
傳產其他        538
觀光休閒娛樂      524
紡織          395
運輸          378
化學工業        291
公用事業        259
食品飲料        232
石化          213
水泥          178
農林漁牧        175
造紙          164
建材居家用品      123
電器電纜         92
航天軍工         54
綜合           33
控股            3
dtype: int64

In [174]:
dict_count={}
dict_idu_list=test_dict_collect_industry_50000
for k in dict_idu_list:
    dict_count[k]=len(dict_idu_list[k])
df=pd.Series(dict_count)
df.sort_values(ascending=False)

科技相關      6835
金融相關      1259
鋼鐵金屬      1082
車輛相關       359
營建地產       334
生技醫療保健     322
百貨通路       321
傳播出版       311
能源         244
電機機械       214
傳產其他       179
觀光休閒娛樂     174
紡織         131
運輸         125
化學工業        96
公用事業        86
食品飲料        77
石化          71
水泥          59
農林漁牧        58
造紙          54
建材居家用品      40
電器電纜        30
航天軍工        18
綜合          11
控股           1
dtype: int64

## Build dict train/test industry:index in all single industry VIP news  (140000 and 10000)

In [4]:
with open("D:3.AutoencoderForArticle/DataSet_vip_single_industry","rb") as f:
    Dataset=pickle.load(f)

In [5]:
Dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154884 entries, 0 to 154883
Data columns (total 3 columns):
guid                           154884 non-null object
title_token_plus_body_token    154884 non-null object
indusrty_tags                  154884 non-null object
dtypes: object(3)
memory usage: 3.5+ MB


#### build dict_collect_industry

In [8]:
np.random.seed(5)
suffleindex=np.random.permutation(len(Dataset))
pick=suffleindex
pick

array([ 15951, 142149,  67130, ...,  20463,  18638,  35683])

In [9]:
dict_collect_industry={i:[] for i in list_industry }
for i,j in Dataset.loc[pick].iterrows():
    for k in dict_collect_industry:
        if k in j[2]:
            dict_collect_industry[k].append(i)
            break

In [11]:
dict_count={}
for k in dict_collect_industry:
    dict_count[k]=len(dict_collect_industry[k])
df=pd.Series(dict_count)
df.sort_values(ascending=False)

科技相關      84507
金融相關      15791
鋼鐵金屬      13147
車輛相關       4551
生技醫療保健     4098
百貨通路       4064
營建地產       4054
傳播出版       3800
能源         2965
電機機械       2542
傳產其他       2294
觀光休閒娛樂     2105
紡織         1700
運輸         1523
化學工業       1190
公用事業       1060
食品飲料        998
石化          994
農林漁牧        746
水泥          735
造紙          717
建材居家用品      590
電器電纜        350
航天軍工        219
綜合          136
控股            8
金屬礦採選         0
其他            0
dtype: int64

#### build train/test_dict_collect_industry

In [17]:
train_dict_collect_industry=dict()
test_dict_collect_industry=dict()
np.random.seed(10)
rate=(154884-10000)/154884
for i in dict_collect_industry:
    induslist=np.random.permutation(dict_collect_industry[i])
    if 0!=len(induslist):
        split=int(np.floor(len(induslist)*rate))
        train_dict_collect_industry[i]=induslist[:split]
        test_dict_collect_industry[i]=induslist[split:]

In [18]:
dict_count={}
dict_idu_list=train_dict_collect_industry
for k in dict_idu_list:
    dict_count[k]=len(dict_idu_list[k])
df=pd.Series(dict_count)
df.sort_values(ascending=False)

科技相關      79050
金融相關      14771
鋼鐵金屬      12298
車輛相關       4257
生技醫療保健     3833
百貨通路       3801
營建地產       3792
傳播出版       3554
能源         2773
電機機械       2377
傳產其他       2145
觀光休閒娛樂     1969
紡織         1590
運輸         1424
化學工業       1113
公用事業        991
食品飲料        933
石化          929
農林漁牧        697
水泥          687
造紙          670
建材居家用品      551
電器電纜        327
航天軍工        204
綜合          127
控股            7
dtype: int64

In [19]:
dict_count={}
dict_idu_list=test_dict_collect_industry
for k in dict_idu_list:
    dict_count[k]=len(dict_idu_list[k])
df=pd.Series(dict_count)
df.sort_values(ascending=False)

科技相關      5457
金融相關      1020
鋼鐵金屬       849
車輛相關       294
生技醫療保健     265
百貨通路       263
營建地產       262
傳播出版       246
能源         192
電機機械       165
傳產其他       149
觀光休閒娛樂     136
紡織         110
運輸          99
化學工業        77
公用事業        69
石化          65
食品飲料        65
農林漁牧        49
水泥          48
造紙          47
建材居家用品      39
電器電纜        23
航天軍工        15
綜合           9
控股           1
dtype: int64

#### Save train_test_dict_collect_industry

In [24]:
with open("D:3.AutoencoderForArticle/train_dict_collect_industry.p","wb") as f:
    pickle.dump(file=f,obj=train_dict_collect_industry)
with open("D:3.AutoencoderForArticle/test_dict_collect_industry.p","wb") as f:
    pickle.dump(file=f,obj=test_dict_collect_industry)

array([0, 3, 4, 1, 2])